In [15]:
import psycopg2
import json
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer

number_of_results = 5

# Database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Load the projects
with open("digital_agriculture_projects.json", "r") as f:
    projects = json.load(f)

# Function to perform the search
def search(query, model, db_config):
    # Encode the query
    query_embedding = model.encode([query])[0]
    query_embedding = [float(value) for value in query_embedding]

    
    # Connect to the database
    with psycopg2.connect(**db_config) as conn:
        c = conn.cursor()

        # Search for the top projects by cosine similarity
        c.execute("""
            SELECT project_id, chunk, embedding <-> %s::VECTOR AS distance
            FROM embeddings
            ORDER BY distance ASC
            LIMIT %s;
        """, (list(query_embedding), number_of_results))
        
        results = c.fetchall()
    
    return results

# Initialize the model
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Define the database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}



In [16]:
#convert this notebook to a python script
!jupyter nbconvert --to script search.ipynb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[NbConvertApp] Converting notebook search.ipynb to script
[NbConvertApp] Writing 2176 bytes to search.py


In [19]:
# Define the query
query = "Main challengues for Climate Smart Agriculture"

# Perform the search
results = search(query, model, db_config)
results[::-1]
# Print the results
for project_id, chunk, distance in results:
    project = next((p for p in projects if ",".join(p["ids"]) == project_id), None)
    print(f"Project ID: {project_id}")
    print(f"Project title: {project['title']}")
    print(f"Project abstract: {project['abstract']}")
    print(f"Relevant snippet: {chunk}")
    print(f"Distance: {distance}")
    print("\n")


Project ID: P154784
Project title: Kenya - Climate Smart Agriculture Project : Environmental Assessment (Vol. 7) : Environmental and Social Impact Assessment for Sertonje Borehole Sub-project, Mugurin Sub Location, Simotwe Location, Kisanana Ward, Mogotio Sub County, Baringo County
Project abstract: The development objective of the Climate Smart Agriculture Project for Kenya is to increase agricultural productivity and build resilience to climate change risks in the targeted smallholder farming and pastoral communities in Kenya, and in the event of an eligible crisis or emergency, to provide immediate and effective response. Suggested measures included sensitization of the community on livestock disease control and prevention and provision of extension, intensify disease and pest surveillance; rehabilitate through reinstatement and tree planting all sites that are being used for construction activities such as camps, materials site (borrow pits and quarries) sites for storage materials